![](./Attention.png)


# Self Attention

In [1]:
import torch
print(torch.__version__)

1.12.1


- Attention Block
- MultiHeadAttention
- Encoder Layer
- Pytorch Official Implementation

In [2]:
device = torch.device('mps')

In [3]:
# X는 8개단어, 각 단어는 3개의 차원, 
X = torch.Tensor(torch.randn(8,3)).to(device)

In [4]:
print(X) # Vectorized Input vector

tensor([[ 0.0675, -0.2713,  0.0818],
        [ 2.2818,  0.8646, -1.4367],
        [-0.4950,  0.2794,  1.2538],
        [ 1.3112,  0.4208, -1.2282],
        [-0.6134,  0.2767, -0.3212],
        [-1.5270, -0.7643,  0.2412],
        [-1.0367,  0.1147,  0.9191],
        [ 0.7185,  0.9433,  1.1128]], device='mps:0')


In [5]:
W_Q = torch.nn.Parameter(torch.Tensor(torch.randn(3,2))).to(device) # Weight
W_K = torch.nn.Parameter(torch.Tensor(torch.randn(3,2))).to(device)
W_V = torch.nn.Parameter(torch.Tensor(torch.randn(3,2))).to(device)

In [7]:
Q = torch.matmul(X, W_Q)
K = torch.mm(X, W_K)
V = X@W_V # @ : broadcast 되는게 좋아서 주로 구현 시 사용하도록 함
X.matmul(W_K)

In [8]:
# Attention Score : Query와 Key 가 잘 맞는 정도 ! 
# Q X KT

# Q@K.T
attention_score = Q.matmul(K.T)

In [9]:
d_k = K.shape[1]  # Scaling Factor
print(d_k)

2


In [18]:
attention_score = attention_score/(d_k**0.5)
# attention_score = attention_score/(math.sqrt(d_k))
# attention_score = attention_score/(torch.sqrt(torch.Tensor(d_k)))

In [28]:
# torch.nn.Softmax  # layer
attention_score =torch.softmax(attention_score, dim=1) # functional

In [30]:
# attention_score[0,:].sum()

attention_score.sum(dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       device='mps:0', grad_fn=<SumBackward1>)

In [32]:
# 구현의효율을 위해 scaling factor 을 Q에 나눠주기도 함. (Q/d_k**2)
# 연산량이 줄어듬 (official code 참고해보면)

Z = attention_score@V # Weighted sum 한 V 완성
print(Z.shape)

torch.Size([8, 2])


In [ ]:
# Attention Block을 병렬적으로 (Attention head 갯수만큼) 쌓여서 동작하는 것 : 
# 1개만 있을 경우 diversity 한정, 다양한 embedding 위해서 (feature space 한정)
# 